<a href="https://colab.research.google.com/github/nagbhushan-1997/Simple-RAG-and-Hybrid-RAG-Implementation/blob/main/Simple_and_Hybrid_RAG_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU "langchain-chroma>=0.1.2"
!pip install langchain_community
!pip install pypdf
!pip install -qU langchain-mistralai
!pip install langchain, langchain_core

ERROR: Invalid requirement: 'langchain,': Expected end or semicolon (after name and no valid version specifier)
    langchain,
             ^


In [ ]:
from google.colab import userdata

# Access the key by its name
mistral_api_key = userdata.get('Mistral_API')

# Use the api_key variable in your API call
if mistral_api_key is not None:
    # Your API call code here
    pass
else:
    print("API key not found in Colab secrets.")

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_mistralai import MistralAIEmbeddings
embeddings = MistralAIEmbeddings(
    model = "mistral-embed",
    api_key=mistral_api_key
)
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
from langchain_core.messages import AIMessage,HumanMessage, SystemMessage


In [ ]:
loader = PyPDFLoader("/content/drive/MyDrive/Data_Sets/RAG_and_AgenticRAG/EcoSprint_Specification_Document.pdf")

In [ ]:
docs = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=256)
splits = text_splitter.split_documents(docs)

In [ ]:
vector_store = Chroma.from_documents(
    documents=splits,
    embedding=embeddings
)

In [ ]:
simple_retriever = vector_store.as_retriever(search_kwrgs = {"k":3})

In [ ]:
from langchain_mistralai import ChatMistralAI

In [ ]:
llm = ChatMistralAI(
    model="mistral-large-latest",
    temperature=0,
    api_key=mistral_api_key

)

In [ ]:
simple_retriever_prompt = ChatPromptTemplate.from_template("""
Answer the question using only the context below.

Context:
{context}

Question:
{question}
""")

In [ ]:
# RAG pipeline (LCEL)
simple_rag_chain = (
    {
        "context": simple_retriever,
        "question": RunnablePassthrough()
    }
    | simple_retriever_prompt
    | llm
    | StrOutputParser()
)

In [ ]:
# Query
response = simple_rag_chain.invoke(
    "Which EcoSprint components have extended warranty?"
)

print(response)

Based on the provided context, the **battery** of the EcoSprint has an extended warranty of **8 years/100,000 miles**, while the **vehicle** itself comes with a **5-year/60,000-mile warranty**.


# Hybrid RAG

In [ ]:
!pip install rank-bm25

In [ ]:
dense_retriever = vector_store.as_retriever(search_kwargs={"k": 3})

In [ ]:
chunks = text_splitter.split_documents(docs)

In [ ]:
from langchain_community.retrievers import BM25Retriever
bm25_retriever = BM25Retriever.from_documents(chunks)
bm25_retriever.k = 3


In [ ]:
def hybrid_retrieve(query: str):
    dense_docs = dense_retriever.invoke(query)
    sparse_docs = bm25_retriever.invoke(query)

    # Deduplicate by content
    seen = set()
    combined = []
    for doc in dense_docs + sparse_docs:
        if doc.page_content not in seen:
            combined.append(doc)
            seen.add(doc.page_content)

    return combined

In [ ]:
llm = ChatMistralAI(
    model="mistral-large-latest",
    temperature=0,
    api_key=mistral_api_key

)

In [ ]:
prompt = ChatPromptTemplate.from_template("""
Answer the question using only the context below.

Context:
{context}

Question:
{question}
""")

In [ ]:
def format_docs(docs):
    return "\n\n".join(d.page_content for d in docs)

rag_chain = (
    {
        "context": lambda q: format_docs(hybrid_retrieve(q)),
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
print(
    rag_chain.invoke(
        "Compare vehicle warranty vs battery warranty limitations"
    )
)


Based on the provided context:

- The **vehicle warranty** covers the EcoSprint for **5 years or 60,000 miles**, whichever comes first.
- The **battery warranty** is longer, covering **8 years or 100,000 miles**, whichever comes first.


In [ ]:
!pip install sentence-transformers


In [ ]:
from sentence_transformers import CrossEncoder

# High-quality re-ranker (fast enough for demos)
reranker = CrossEncoder(
    "cross-encoder/ms-marco-MiniLM-L-6-v2"
)

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

In [ ]:
def hybrid_retrieve_with_rerank(query: str, top_k: int = 3):
    # Step 1: Retrieve candidates
    dense_docs = dense_retriever.invoke(query)
    sparse_docs = bm25_retriever.invoke(query)

    # Step 2: Merge & deduplicate
    seen = set()
    candidates = []
    for doc in dense_docs + sparse_docs:
        if doc.page_content not in seen:
            candidates.append(doc)
            seen.add(doc.page_content)

    # Step 3: Prepare pairs for cross-encoder
    pairs = [(query, doc.page_content) for doc in candidates]

    # Step 4: Score using cross-encoder
    scores = reranker.predict(pairs)

    # Step 5: Sort by score
    reranked = sorted(
        zip(candidates, scores),
        key=lambda x: x[1],
        reverse=True
    )

    # Step 6: Select top-k
    top_docs = [doc for doc, _ in reranked[:top_k]]

    return top_docs


In [ ]:
def format_docs(docs):
    return "\n\n".join(d.page_content for d in docs)

rag_chain = (
    {
        "context": lambda q: format_docs(
            hybrid_retrieve_with_rerank(q, top_k=3)
        ),
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

print(
    rag_chain.invoke(
        "Which EcoSprint components have extended warranty?"
    )
)


Based on the provided context, the **EcoSprint battery** has an extended warranty of **8 years or 100,000 miles**, while the **vehicle** itself has a **5-year/60,000-mile warranty**.
